<a href="https://colab.research.google.com/github/nadaYossef/Boston-Seattle-Price-Prediction/blob/main/grad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow opencv-python numpy gTTS pygame

In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from gtts import gTTS
import pygame

# Path to the dataset
video_dir = 'path_to_your_videos/'  # Replace with the path to the video dataset

# Preprocess video frames
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))  # Resize to match VGG16 input
        frame = frame / 255.0  # Normalize to [0,1]
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Load videos and labels (Assuming you have a mapping of videos to labels)
def load_dataset(video_dir):
    labels = []  # To store corresponding labels
    video_data = []  # To store processed videos
    for video_file in os.listdir(video_dir):
        if video_file.endswith('.mp4'):  # Modify according to your file extension
            video_path = os.path.join(video_dir, video_file)
            frames = preprocess_video(video_path)
            video_data.append(frames)
            # Assuming label is in the filename or similar logic
            label = video_file.split('_')[0]  # Modify as needed
            labels.append(label)
    return np.array(video_data), np.array(labels)

# Load dataset
video_data, labels = load_dataset(video_dir)

# Create a mapping from labels to indices
label_to_index = {label: index for index, label in enumerate(np.unique(labels))}
index_to_label = {index: label for label, index in label_to_index.items()}
num_classes = len(label_to_index)

# Convert labels to one-hot encoding
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels_one_hot = lb.fit_transform(labels)

# Load VGG16 Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(video_data, labels_one_hot, epochs=10, batch_size=32)  # Adjust epochs and batch size as needed

# Save the model
model.save('asl_video_model.h5')

# Function to map predicted labels to words/sentences
def label_to_word(predicted_label):
    return index_to_label[predicted_label]

# Function to play audio using pygame
def play_audio(text):
    tts = gTTS(text=text, lang='en')
    tts.save("temp.mp3")
    pygame.mixer.init()
    pygame.mixer.music.load("temp.mp3")
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():  # Wait for the audio to finish playing
        continue

# Function for real-time prediction from webcam
def predict_from_webcam():
    cap = cv2.VideoCapture(0)  # Capture from webcam
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame
        frame_resized = cv2.resize(frame, (224, 224))
        frame_normalized = frame_resized / 255.0  # Normalize
        processed_frame = np.expand_dims(frame_normalized, axis=0)  # Add batch dimension

        # Make predictions
        predictions = model.predict(processed_frame)
        predicted_label = np.argmax(predictions)

        # Convert predicted label to corresponding word
        translated_word = label_to_word(predicted_label)

        # Display prediction on the frame
        cv2.putText(frame, translated_word, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Use text-to-speech to say the word aloud
        play_audio(translated_word)

        # Show the frame
        cv2.imshow('Webcam', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the prediction function
predict_from_webcam()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_videos/'